## Import all of used library 
is needed to run all of the cells in this section

In [1]:
# To remove all warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
import zipfile
import pandas as pd
import requests
import random
import calendar
import time
import pickle
import tempfile
import joblib

from PIL import Image

from tqdm import tqdm

from shutil import move
from shutil import copy
from shutil import make_archive
from shutil import rmtree

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import MobileNet, MobileNetV2, VGG16, EfficientNetB0, InceptionV3, \
                                           VGG19, Xception, DenseNet121, DenseNet201, ResNet152V2, EfficientNetB5
from tensorflow.keras.layers import Input, AveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import load_model
from tensorflow import lite, cast, float32, saved_model, keras
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

from imblearn.over_sampling import RandomOverSampler 

In [2]:
# To mounting Google Drive

def google_drive_mount(mounting=False):
  if mounting:
    drive.mount('/content/drive')

In [3]:
# Download the file

def download_file(urls, unzip=True, remove_after_unzip=False, google_drive=False, filename='tmp', ext='zip'):

  url = urls
  CHUNK_SIZE = 32768
  file_tmp = f'{filename}.{ext}'

  # If Downloading from Google Drive
  if google_drive:

    # Get the token
    def get_confirm_token(response):
      
      for key, value in response.cookies.items():
        if key.startswith('download_warning'):
          return value
      return None
    
    url_raw = 'https://docs.google.com/uc?export=download'
    file_id = url.split('/')[-2] # Split the file id
    session = requests.Session() # Get session
    response = session.get(url_raw, params={'id': file_id}, stream=True) # Get the response
    token = get_confirm_token(response) # Get the token

    # If there is token
    if token:
      response = session.get(url_raw, params={'id': file_id, 'confirm': token}, stream=True) # Get the response
    
    # Save the response as zip file
    with open(f'{os.getcwd()}/{file_tmp}', 'wb') as f:
      for chunk in response.iter_content(CHUNK_SIZE):
        if chunk:
          f.write(chunk)
  
  # If not from Google Drive
  else:
    filedata = get_file(origin=url, fname=os.path.join(os.getcwd(), file_tmp)) # Get the response from URL

  # Extract the zip files
  if unzip:
    local_zip = f'{os.getcwd()}/{file_tmp}'
    zip_ref = zipfile.ZipFile(local_zip, 'r') # Extract 
    zip_ref.extractall(f'{os.getcwd()}/') # Extract
    zip_ref.close()
    # Remove the zip file after unzip
    if remove_after_unzip:
      os.remove(f'{os.getcwd()}/{file_tmp}')  
  
  # If not a zip file
  else:
    return f'{os.getcwd()}/{file_tmp}'

In [ ]:
# Load the saved models/checkpoints models from Google Drive

def load_models(url, summary=False, filenames='tmp'):
  model_download = download_file(url, unzip=False, remove_after_unzip=True, google_drive=True, ext="h5", filename=filenames)  
  model = load_model(model_download)
  if summary:
    model.summary()
  return model

In [ ]:
links = 'https://drive.google.com/file/d/1-0ODyEWBJcERmvXXM5Ejk_VoxV7gI6SW/view?usp=sharing'
model = load_models(links)

# Convert Keras Model to Tensorflow SavedModel
Purposed in this section to convert checkpoint model (keras *.h5 format) to SavedModel Tensorflow format, in order you want to deploy it to web or Google Cloud or something that needed Tensorflow SavedModel

In [ ]:
# Load the saved models/checkpoints models from Google Drive

def KerastoSavedModel(links, path='/content/drive/MyDrive/', 
                      make_zip=False, move_to_drive=True, name='models', 
                      ext='zip', raw_model=False):

    # Load model from Keras
    def load_models(url, summary=False, filenames='tmp'):    
        model_download = download_file(url, unzip=False, remove_after_unzip=True, google_drive=True, ext="h5", filename=filenames)  
        model = load_model(model_download)    
        if summary:
            model.summary()
        return model
    model = load_models(links)

    # Convert it to SavedModel
    saved_model.save(model, f'{os.getcwd()}/{name}')

    # Store it on zip files
    if make_zip:
        make_archive(name, ext, f'{os.getcwd()}/{name}') # Create the new zip file
    
    # Move it to drive
    if move_to_drive:
        google_drive_mount(mounting=True) # Mount Google Drive
        
        if not os.path.exists(path):
            os.mkdir(path)

        copy(f'{os.getcwd()}/{name}.{ext}', path) # Move zip file to Google Drive
        print(f'Saved model zip to {path}')
        
        if raw_model: # Move raw model to Google Drive
            copy(f'{os.getcwd()}/{name}', path)
            print(f'Saved model raw to {path}')

In [ ]:
links = 'https://drive.google.com/file/d/1-0ODyEWBJcERmvXXM5Ejk_VoxV7gI6SW/view?usp=sharing'

KerastoSavedModel(links, make_zip=True, move_to_drive=True, raw_model=True, name='test_model',
                  path='/content/drive/MyDrive/fix-datasets-new/test')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: /content/test_model/assets
Mounted at /content/drive
Saved model zip to /content/drive/MyDrive/fix-datasets-new/test
Saved model raw to /content/drive/MyDrive/fix-datasets-new/test


# Convert to TFLite
Purposed in this section to convert checkpoint model (keras *.h5 format) to Tensorflow lite format, in order you want to deploy it to web, embedded, mobile or microcontroller devices

In [ ]:
def to_tflite(links, tflite_filename='converted_model', optimizations='size',
              move_to_drive=False, path='/content/drive/MyDrive/'):
    
    # Load model from Keras
    def load_models(url, summary=False, filenames='tmp'):    
        model_download = download_file(url, unzip=False, remove_after_unzip=True, google_drive=True, ext="h5", filename=filenames)  
        model = load_model(model_download)    
        if summary:
            model.summary()
        return model
    model = load_models(links)
    
    converter = lite.TFLiteConverter.from_keras_model(model)
    
    if optimizations in ['size', 'SIZE']:
        converter.optimizations = [lite.Optimize.OPTIMIZE_FOR_SIZE]
    
    if optimizations in ['latency', 'LATENCY']:
        converter.optimizations = [lite.Optimize.OPTIMIZE_FOR_LATENCY]
    
    tflite_model = converter.convert()
    
    with open(f'{os.getcwd()}/{tflite_filename}.tflite', 'wb') as f:
        f.write(tflite_model)

    if move_to_drive:
        google_drive_mount(mounting=True) # Mount Google Drive
        
        if not os.path.exists(path):
            os.mkdir(path)

        copy(f'{os.getcwd()}/{tflite_filename}.tflite', path) # Move zip file to Google Drive
        print(f'Saved tflite to {path}')
    
    return f'{tflite_filename}.tflite'

In [ ]:
links = 'https://drive.google.com/file/d/1-0ODyEWBJcERmvXXM5Ejk_VoxV7gI6SW/view?usp=sharing' # Load Keras Models or SavedModel

tflite_filename = to_tflite(links, move_to_drive=True, tflite_filename='test',
                            path='/content/drive/MyDrive/fix-datasets-new/test')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/tmpe_9j8atz/assets


INFO:tensorflow:Assets written to: /tmp/tmpe_9j8atz/assets


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved tflite to /content/drive/MyDrive/fix-datasets-new/test


# Pickle the model


In [4]:
# Load the saved models/checkpoints models from Google Drive

def load_models(url, summary=False, filenames='tmp'):
    
    model_download = download_file(url, unzip=False, remove_after_unzip=True, google_drive=True, ext="h5", filename=filenames)  
    
    model = load_model(model_download)
    
    if summary:
        model.summary()
    
    return model

In [5]:
links = 'https://drive.google.com/file/d/1-0ODyEWBJcERmvXXM5Ejk_VoxV7gI6SW/view?usp=sharing'
model = load_models(links)

In [6]:
def unpack(model, training_config, weights):
    restored_model = deserialize(model)
    if training_config is not None:
        restored_model.compile(
            **saving_utils.compile_args_from_training_config(
                training_config
            )
        )
    restored_model.set_weights(weights)
    return restored_model

# Hotfix function
def make_keras_picklable():
    def __reduce__(self):
        model_metadata = saving_utils.model_metadata(self)
        training_config = model_metadata.get("training_config", None)
        model = serialize(self)
        weights = self.get_weights()
        return (unpack, (model, training_config, weights))
    cls = keras.Model
    cls.__reduce__ = __reduce__

# Source: https://github.com/tensorflow/tensorflow/issues/34697

In [7]:
def convert_pickle(model, model_name='model', ext='pkl', save_to_drive=False,
                   path_to_drive='/content/drive/MyDrive/fix-datasets/saved_model'):
    
    make_keras_picklable() # Call picklabel function
    
    if ext is 'pkl':
        with open(f'{model_name}.{ext}', 'wb') as f:
            pickle.dump(model, f)
        print(f'Converting {model_name}.{ext} succesfully')
    
    if ext is 'joblib':
        with open(f'{model_name}.{ext}', 'wb') as f:
            joblib.dump(model, f)
        print(f'Converting {model_name}.{ext} succesfully')
    
    if save_to_drive:
        google_drive_mount(mounting=True) 
        copy(f'{model_name}.{ext}', path_to_drive) 

In [8]:
convert_pickle(model)
convert_pickle(model, ext='joblib')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Converting model.pkl succesfully
Converting model.joblib succesfully
